In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/data/home/marmot/camtrap/PyCharm/CameraTraps-benchmark')

In [3]:
import json
from collections import OrderedDict
from copy import deepcopy

from data_management.cct_json_utils import CameraTrapJsonUtils

# Load original DBs and splits

## Caltech

In [4]:
with open('/beaver_disk/camtrap/caltech/original/CaltechCameraTrapsSplits_v0.json') as f:
    cct_splits = json.load(f)

In [5]:
for loc in cct_splits['splits']['eccv_train']:
    assert loc in set(cct_splits['splits']['train'])

In [6]:
for name, split in cct_splits['splits'].items():
    print(name, len(split))
    print(name, len(set(split)))

eccv_val 10
eccv_val 10
val 40
val 40
eccv_train 30
eccv_train 10
train 120
train 100


Using version 2.1 of the CaltechCameraTraps DB (we are only using the CCT-20 locations anyways)

In [7]:
with open('/beaver_disk/camtrap/caltech/original/CaltechCameraTraps_v2.1.json') as f:
    cct_db = json.load(f)

In [8]:
len(cct_db['images'])
len(cct_db['annotations'])

244497

244577

In [9]:
num_unlabeled = 0
for a in cct_db['annotations']:
    if a['category_id'] == 999:
        num_unlabeled += 1
num_unlabeled

42087

In [10]:
cct_db['images'][100]
cct_db['annotations'][100]

{'date_captured': '2012-06-02 07:05:58',
 'file_name': '596a4e34-23d2-11e8-a6a3-ec086b02610b.jpg',
 'frame_num': 2,
 'height': 1494,
 'id': '596a4e34-23d2-11e8-a6a3-ec086b02610b',
 'location': '46',
 'rights_holder': 'Justin Brown',
 'seq_id': '6f19090c-5567-11e8-8ff6-dca9047ef277',
 'seq_num_frames': 3,
 'width': 2048}

{'category_id': 9,
 'id': '2a5492ce-cbf1-11e8-819c-970a9450cdbc',
 'image_id': '596a4e34-23d2-11e8-a6a3-ec086b02610b'}

In [11]:
len(cct_db['categories'])
sorted(cct_db['categories'], key=lambda x: x['id'])

23

[{'id': 1, 'name': 'opossum'},
 {'id': 3, 'name': 'raccoon'},
 {'id': 5, 'name': 'squirrel'},
 {'id': 6, 'name': 'bobcat'},
 {'id': 7, 'name': 'skunk'},
 {'id': 8, 'name': 'dog'},
 {'id': 9, 'name': 'coyote'},
 {'id': 10, 'name': 'rabbit'},
 {'id': 11, 'name': 'bird'},
 {'id': 14, 'name': 'lizard'},
 {'id': 16, 'name': 'cat'},
 {'id': 21, 'name': 'badger'},
 {'id': 30, 'name': 'empty'},
 {'id': 33, 'name': 'car'},
 {'id': 34, 'name': 'deer'},
 {'id': 37, 'name': 'cow'},
 {'id': 39, 'name': 'pig'},
 {'id': 40, 'name': 'mountain_lion'},
 {'id': 51, 'name': 'fox'},
 {'id': 66, 'name': 'bat'},
 {'id': 97, 'name': 'insect'},
 {'id': 99, 'name': 'rodent'},
 {'id': 999, 'name': 'unlabeled_animal'}]

In [7]:
with open('/beaver_disk/camtrap/caltech/original/caltech_bboxes_20190904.json') as f:
    cct_boxes = json.load(f)

In [8]:
len(cct_boxes['images'])
len(cct_boxes['annotations'])

63102

66406

In [9]:
cct_boxes['images'][100]
cct_boxes['annotations'][100]

{'date_captured': '2011-09-21 13:15:49',
 'file_name': '59421138-23d2-11e8-a6a3-ec086b02610b.jpg',
 'frame_num': 1,
 'height': 1494,
 'id': '59421138-23d2-11e8-a6a3-ec086b02610b',
 'location': '88',
 'rights_holder': 'Justin Brown',
 'seq_id': '6f13fdb0-5567-11e8-80f5-dca9047ef277',
 'seq_num_frames': 3,
 'width': 2048}

{'bbox': [883.2, 612.693359375, 320.8533593750001, 358.4],
 'category_id': 1,
 'id': '2a551c6c-cbf1-11e8-819c-970a9450cdbc',
 'image_id': '59421138-23d2-11e8-a6a3-ec086b02610b'}

In [10]:
len(cct_boxes['categories'])
sorted(cct_boxes['categories'], key=lambda x: x['id'])

4

[{'id': 1, 'name': 'animal'},
 {'id': 2, 'name': 'person'},
 {'id': 3, 'name': 'group'},
 {'id': 4, 'name': 'vehicle'}]

## SS and especially season 1

In [4]:
with open('/beaver_disk/camtrap/ss_season1/original/SnapshotSerengetiSplits_v0.json') as f:
    ss_splits = json.load(f)

In [5]:
len(ss_splits['splits']['train'])
len(ss_splits['splits']['val'])

179

46

In [6]:
with open('/beaver_disk/camtrap/ss_season1/original/SnapshotSerengetiS01.json') as f:
    ss_season1_db = json.load(f)

In [7]:
len(ss_season1_db['images'])
len(ss_season1_db['annotations'])

411414

412858

In [8]:
ss_season1_db['images'][200]

{'corrupt': False,
 'datetime': '2010-09-19 11:51:58',
 'file_name': 'S1/H10/H10_R1/S1_H10_R1_PICT0664.JPG',
 'frame_num': 3,
 'height': 1536,
 'id': 'S1/H10/H10_R1/S1_H10_R1_PICT0664',
 'location': 'H10',
 'seq_id': 'SER_S1#H10#1#222',
 'seq_num_frames': 3,
 'width': 2048}

In [9]:
for i in ss_season1_db['images']:
    assert i['file_name'].startswith('S1/')

In [10]:
len(ss_season1_db['categories'])
sorted(ss_season1_db['categories'], key=lambda x: x['id'])

61

[{'id': 0, 'name': 'empty'},
 {'id': 1, 'name': 'human'},
 {'id': 2, 'name': 'gazellegrants'},
 {'id': 3, 'name': 'reedbuck'},
 {'id': 4, 'name': 'dikdik'},
 {'id': 5, 'name': 'zebra'},
 {'id': 6, 'name': 'porcupine'},
 {'id': 7, 'name': 'gazellethomsons'},
 {'id': 8, 'name': 'hyenaspotted'},
 {'id': 9, 'name': 'warthog'},
 {'id': 10, 'name': 'impala'},
 {'id': 11, 'name': 'elephant'},
 {'id': 12, 'name': 'aardvark'},
 {'id': 13, 'name': 'giraffe'},
 {'id': 14, 'name': 'mongoose'},
 {'id': 15, 'name': 'buffalo'},
 {'id': 16, 'name': 'hartebeest'},
 {'id': 17, 'name': 'guineafowl'},
 {'id': 18, 'name': 'wildebeest'},
 {'id': 19, 'name': 'leopard'},
 {'id': 20, 'name': 'ostrich'},
 {'id': 21, 'name': 'lionfemale'},
 {'id': 22, 'name': 'koribustard'},
 {'id': 23, 'name': 'otherbird'},
 {'id': 24, 'name': 'cheetah'},
 {'id': 25, 'name': 'honeybadger'},
 {'id': 26, 'name': 'bushbuck'},
 {'id': 27, 'name': 'jackal'},
 {'id': 28, 'name': 'aardwolf'},
 {'id': 29, 'name': 'hippopotamus'},
 {'id

In [11]:
# ! Updated bbox json with insect and distant birds excluded
with open('/beaver_disk/camtrap/ss_season1/original/SnapshotSerengetiBboxes_20190903.json') as f:
    ss_boxes = json.load(f)

In [12]:
len(ss_boxes['images'])
len(ss_boxes['annotations'])

82938

146359

In [13]:
ss_boxes['categories']

[{'id': 1, 'name': 'animal'},
 {'id': 2, 'name': 'person'},
 {'id': 3, 'name': 'group'},
 {'id': 4, 'name': 'vehicle'}]

In [14]:
ss_boxes['annotations'][100]

{'bbox': [0, 0, 1057.5135192871116, 802.8107910156289],
 'category_id': 1,
 'id': 'IXQwi1533020694359',
 'image_id': 'S1/E06/E06_R1/S1_E06_R1_PICT0018'}

In [15]:
ss_boxes['info']

{'contributor': 'Sara Beery, this version updated by Siyu Yang',
 'date_created': '2019-09-03',
 'description': 'Reprocessed bounding box annotations for Snapshot Serengeti seasons 1 to 6, with remaining small insect and distant bird bboxes smaller than 400 sq pixel excluded manually.',
 'version': '20190903',
 'year': 2019}

In [16]:
ss_boxes_season1 = {
    'images': [i for i in ss_boxes['images'] if i['id'].startswith('S1/')],
    'annotations': [a for a in ss_boxes['annotations'] if 'image_id' in a and a['image_id'].startswith('S1/')],
    'categories': ss_boxes['categories'],
    'info': {
        'contributor': 'SMB',
        'date_created': '2019-04-14',
        'description': 'Reprocessed bounding box annotations for Snapshot Serengeti season 1. Based on Bbox DB version 20190903.',
        'version': '20190903',
        'year': 2019
    }
}

In [17]:
len(ss_boxes_season1['images'])
len(ss_boxes_season1['annotations'])

12377

19387

In [18]:
with open('/beaver_disk/camtrap/ss_season1/benchmark/SnapshotSerengetiBboxesS01_20190903.json', 'w') as f:
    json.dump(ss_boxes_season1, f, indent=1)

# Separate out DB for train and val

For both images and bounding boxes DBs since categories have different meaning in each set.

## CCT

Only using the CCT-20 locations!

In [11]:
cct_train_split = set(cct_splits['splits']['eccv_train'])
cct_val_split = set(cct_splits['splits']['eccv_val'])
len(cct_train_split)
len(cct_val_split)

10

10

There are some locations such as '120' that are repeated in the training split (but `eccv_train` is fine).

### CCT images DBs

In [17]:
cct_train_db = CameraTrapJsonUtils.get_entries_from_locations(cct_db, cct_train_split)

Original DB has 244497 image and 244577 annotation entries.
New DB has 32865 image and 32873 annotation entries.


In [18]:
cct_train_db['categories'] = cct_db['categories']
cct_train_db['info'] = deepcopy(cct_db['info'])

In [19]:
cct_train_db['info']['description'] = 'Training split of the ECCV subset of ' + cct_train_db['info']['description']
cct_train_db['info']['description']

'Training split of the ECCV subset of Caltech Camera Traps: camera trap images collected from the NPS and the USGS with help from Justin Brown and Erin Boydston'

In [21]:
cct_train_db = CameraTrapJsonUtils.order_db_keys(cct_train_db)
with open('/beaver_disk/camtrap/caltech/benchmark/cct-20/CaltechCameraTraps-20_v2.1_train.json', 'w') as f:
    json.dump(cct_train_db, f, indent=1)

In [17]:
cct_val_db = CameraTrapJsonUtils.get_entries_from_locations(cct_db, cct_val_split)

Original DB has 244497 image and 244577 annotation entries.
New DB has 25000 image and 25070 annotation entries.


In [22]:
cct_val_db['categories'] = cct_db['categories']
cct_val_db['info'] = deepcopy(cct_db['info'])

In [23]:
cct_val_db['info']['description'] = 'Val split of the ECCV subset of ' + cct_val_db['info']['description']
cct_val_db['info']['description']

'Val split of the ECCV subset of Caltech Camera Traps: camera trap images collected from the NPS and the USGS with help from Justin Brown and Erin Boydston'

In [25]:
cct_val_db = CameraTrapJsonUtils.order_db_keys(cct_val_db)
with open('/beaver_disk/camtrap/caltech/benchmark/cct-20/CaltechCameraTraps-20_v2.1_val.json', 'w') as f:
    json.dump(cct_val_db, f, indent=1)

In [35]:
cct_train_db['categories']

[{'id': 6, 'name': 'bobcat'},
 {'id': 1, 'name': 'opossum'},
 {'id': 30, 'name': 'empty'},
 {'id': 9, 'name': 'coyote'},
 {'id': 3, 'name': 'raccoon'},
 {'id': 11, 'name': 'bird'},
 {'id': 8, 'name': 'dog'},
 {'id': 16, 'name': 'cat'},
 {'id': 5, 'name': 'squirrel'},
 {'id': 10, 'name': 'rabbit'},
 {'id': 7, 'name': 'skunk'},
 {'id': 14, 'name': 'lizard'},
 {'id': 99, 'name': 'rodent'},
 {'id': 21, 'name': 'badger'},
 {'id': 34, 'name': 'deer'},
 {'id': 37, 'name': 'cow'},
 {'id': 33, 'name': 'car'},
 {'id': 51, 'name': 'fox'},
 {'id': 39, 'name': 'pig'},
 {'id': 40, 'name': 'mountain_lion'},
 {'id': 66, 'name': 'bat'},
 {'id': 97, 'name': 'insect'},
 {'id': 999, 'name': 'unlabeled_animal'}]

In [33]:
empty_train = []
for i in cct_train_db['annotations']:
    if i['category_id'] == 30:
        empty_train.append(i)

### CCT bboxes DBs

The `location` field in the bbox DB are int instead of str...

In [13]:
cct_train_boxes = CameraTrapJsonUtils.get_entries_from_locations(cct_boxes, cct_train_split)

cct_train_boxes['categories'] = cct_boxes['categories']
cct_train_boxes['info'] = deepcopy(cct_boxes['info'])

Original DB has 63102 image and 66406 annotation entries.
New DB has 26460 image and 27644 annotation entries.


In [14]:
cct_train_boxes['categories'] = cct_boxes['categories']
cct_train_boxes['info'] = deepcopy(cct_boxes['info'])

In [15]:
cct_train_boxes['info']['description'] = 'Training split of the ECCV subset of ' + cct_train_boxes['info']['description']
cct_train_boxes['info']['description']

'Training split of the ECCV subset of Bounding box annotations for 63,102 images from Caltech Camera Traps, covering all classes and locations.  Contains all annotations for CCT - 20, the 20-location dataset used in the ECCV18 paper "Recognition in Terra Incognita," as well as additional annotations collected by MS AI for Earth. Version 20190904 has the categories normed to the four detector categories.'

In [16]:
cct_train_boxes = CameraTrapJsonUtils.order_db_keys(cct_train_boxes)
with open('/beaver_disk/camtrap/caltech/benchmark/cct-20/caltech-20_bboxes_20190904_train.json', 'w') as f:
    json.dump(cct_train_boxes, f, indent=1)

In [17]:
cct_val_boxes = CameraTrapJsonUtils.get_entries_from_locations(cct_boxes, cct_val_split)

cct_val_boxes['categories'] = cct_boxes['categories']
cct_val_boxes['info'] = deepcopy(cct_boxes['info'])

cct_val_boxes['info']['description'] = 'Val split of the ECCV subset of ' + cct_val_boxes['info']['description']
cct_val_boxes['info']['description']

cct_val_boxes = CameraTrapJsonUtils.order_db_keys(cct_val_boxes)
with open('/beaver_disk/camtrap/caltech/benchmark/cct-20/caltech-20_bboxes_20190904_val.json', 'w') as f:
    json.dump(cct_val_boxes, f, indent=1)

Original DB has 63102 image and 66406 annotation entries.
New DB has 19566 image and 20459 annotation entries.


'Val split of the ECCV subset of Bounding box annotations for 63,102 images from Caltech Camera Traps, covering all classes and locations.  Contains all annotations for CCT - 20, the 20-location dataset used in the ECCV18 paper "Recognition in Terra Incognita," as well as additional annotations collected by MS AI for Earth. Version 20190904 has the categories normed to the four detector categories.'

## SS Season 1

In [19]:
ss_train_split = set(ss_splits['splits']['train'])
ss_val_split = set(ss_splits['splits']['val'])
len(ss_train_split)
len(ss_val_split)

179

46

### SS Season 1 images DB

In [101]:
ss1_train_db = CameraTrapJsonUtils.get_entries_from_locations(ss_season1_db, ss_train_split)

ss1_train_db['categories'] = ss_season1_db['categories']
ss1_train_db['info'] = deepcopy(ss_season1_db['info'])

ss1_train_db['info']['description'] = 'Train split of ' + ss1_train_db['info']['description']
ss1_train_db['info']['description']

ss1_train_db = CameraTrapJsonUtils.order_db_keys(ss1_train_db)
with open('/beaver_disk/camtrap/ss_season1/benchmark/SnapshotSerengetiS01_train.json', 'w') as f:
    json.dump(ss1_train_db, f, indent=1)

Original DB has 411414 image and 412858 annotation entries.
New DB has 314591 image and 315605 annotation entries.


'Train split of Camera trap data from the Snapshot Serengeti program, season 1'

In [20]:
ss1_val_db = CameraTrapJsonUtils.get_entries_from_locations(ss_season1_db, ss_val_split)

ss1_val_db['categories'] = ss_season1_db['categories']
ss1_val_db['info'] = deepcopy(ss_season1_db['info'])

ss1_val_db['info']['description'] = 'Val split of ' + ss1_val_db['info']['description']
ss1_val_db['info']['description']

ss1_val_db = CameraTrapJsonUtils.order_db_keys(ss1_val_db)
with open('/beaver_disk/camtrap/ss_season1/benchmark/SnapshotSerengetiS01_val.json', 'w') as f:
    json.dump(ss1_val_db, f, indent=1)

Original DB has 411414 image and 412858 annotation entries.
New DB has 96823 image and 97253 annotation entries.


'Val split of Camera trap data from the Snapshot Serengeti program, season 1'

### SS Season 1 bboxes DB

In [21]:
ss1_train_boxes = CameraTrapJsonUtils.get_entries_from_locations(ss_boxes_season1, ss_train_split)

ss1_train_boxes['categories'] = ss_boxes_season1['categories']
ss1_train_boxes['info'] = deepcopy(ss_boxes_season1['info'])

ss1_train_boxes['info']['description'] = 'Train split of ' + ss1_train_boxes['info']['description']
ss1_train_boxes['info']['description']

ss1_train_boxes = CameraTrapJsonUtils.order_db_keys(ss1_train_boxes)
with open('/beaver_disk/camtrap/ss_season1/benchmark/SnapshotSerengetiBboxesS01_20190903_train.json', 'w') as f:
    json.dump(ss1_train_boxes, f, indent=1)

Original DB has 12377 image and 19387 annotation entries.
New DB has 9244 image and 14949 annotation entries.


'Train split of Reprocessed bounding box annotations for Snapshot Serengeti season 1. Based on Bbox DB version 20190903.'

In [22]:
ss1_val_boxes = CameraTrapJsonUtils.get_entries_from_locations(ss_boxes_season1, ss_val_split)

ss1_val_boxes['categories'] = ss_boxes_season1['categories']
ss1_val_boxes['info'] = deepcopy(ss_boxes_season1['info'])

ss1_val_boxes['info']['description'] = 'Val split of ' + ss1_train_boxes['info']['description']
ss1_val_boxes['info']['description']

ss1_val_boxes = CameraTrapJsonUtils.order_db_keys(ss1_val_boxes)
with open('/beaver_disk/camtrap/ss_season1/benchmark/SnapshotSerengetiBboxesS01_20190903_val.json', 'w') as f:
    json.dump(ss1_val_boxes, f, indent=1)

Original DB has 12377 image and 19387 annotation entries.
New DB has 3133 image and 4438 annotation entries.


'Val split of Train split of Reprocessed bounding box annotations for Snapshot Serengeti season 1. Based on Bbox DB version 20190903.'